In [1]:
import torch
import numpy as np
import genesis as gs

from scipy.spatial.transform import Rotation as R

[I 01/03/25 11:44:04.726 73600] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
########################## init ##########################
gs.init(backend=gs.gpu, logging_level="error")

In [3]:
file_path = '../../genesis/assets/xml/x1/mjcf/xyber_x1_flat.xml'
# file_path = 'xml/franka_emika_panda/panda.xml'
box_path = '../../genesis/assets/xml/x1/mjcf/environment/box.xml'
box_size = 0.2
box_pos = (0.3, 0.0, 1.0)
t = 2

In [4]:
########################## create a scene and entities ##########################
scene = gs.Scene(show_viewer=True, show_FPS=False)
plane = scene.add_entity(gs.morphs.Plane())
robot = scene.add_entity(
    gs.morphs.MJCF(file  = file_path,
                   pos   = (0.0, 0.0, 0.65),
                   euler = (0, 0, 0),),
)
box = scene.add_entity(gs.morphs.Box(pos   = box_pos,
                                     size  = (box_size, box_size*2, box_size/2),
                                     fixed = False,
                                     collision=True,))
# box = scene.add_entity(
#     gs.morphs.MJCF(file  = box_path,
#                    pos   = box_pos,
#                    euler = (0, 0, 0),),
# )
table = scene.add_entity(gs.morphs.Box(pos   = (box_pos[0], box_pos[1], (box_pos[2]-box_size)/2),
                                       size  = (box_size*2, box_size/2, (box_pos[2]-box_size/2)),
                                       fixed = True,
                                       collision=True,))

########################## build ##########################
scene.build()
state = scene.get_state()

In [5]:
robot

────────────────────── <gs.RigidEntity> ──────────────────────
                'n_qs': <int>: 14
              'n_dofs': <int>: 14
             'n_links': <int>: 30
             'n_geoms': <int>: 14
             'n_cells': <numpy.int64>: 1244460
             'n_verts': <int>: 3512
             'n_faces': <int>: 6968
             'n_edges': <int>: 10452
            'n_joints': <int>: 30
            'n_vgeoms': <int>: 38
            'n_vverts': <int>: 379162
            'n_vfaces': <int>: 769092
               'q_end': <int>: 14
             'q_start': <int>: 0
            'is_built': <bool>: True
             'dof_end': <int>: 14
           'dof_start': <int>: 0
                 'idx': <int>: 1
                 'sim': <gs.Simulator>
                 'uid': <gs.UID>('cdaccf1-4a7114b21a76ea20c0fc23ba1')
           'base_link': <gs.RigidLink>: <b1e16bc>, name: 'x1-body', idx: 1
          'base_joint': <gs.RigidJoint>: <fff2cb2>, name: 'x1-body_joint', idx: 1, type: <FIXED: 0>
       'base_

In [6]:
# x1-body floating_base <FREE: 4>: dof 6
# arms dof: 14
# legs dof: 12 --> 0

print(f'number of links: {robot.n_links}')
for link in robot.links:
    print(f'{link.idx} {link.name} {link.joint.name} {link.joint.type}')

number of links: 30
1 x1-body x1-body_joint <FIXED: 0>
2 body_yaw body_yaw_joint <FIXED: 0>
3 left_hip_pitch_link left_hip_pitch_link_joint <FIXED: 0>
4 right_hip_pitch_link right_hip_pitch_link_joint <FIXED: 0>
5 body_roll body_roll_joint <FIXED: 0>
6 left_hip_roll_link left_hip_roll_link_joint <FIXED: 0>
7 right_hip_roll_link right_hip_roll_link_joint <FIXED: 0>
8 body_pitch body_pitch_joint <FIXED: 0>
9 left_hip_yaw_link left_hip_yaw_link_joint <FIXED: 0>
10 right_hip_yaw_link right_hip_yaw_link_joint <FIXED: 0>
11 left_shoulder_pitch_link left_shoulder_pitch <REVOLUTE: 1>
12 right_shoulder_pitch_link right_shoulder_pitch <REVOLUTE: 1>
13 left_knee_pitch_link left_knee_pitch_link_joint <FIXED: 0>
14 right_knee_pitch_link right_knee_pitch_link_joint <FIXED: 0>
15 left_shoulder_roll_link left_shoulder_roll <REVOLUTE: 1>
16 right_shoulder_roll_link right_shoulder_roll <REVOLUTE: 1>
17 left_ankle_pitch_link left_ankle_pitch_link_joint <FIXED: 0>
18 right_ankle_pitch_link right_ankle_pit

In [7]:
########################## joints ##########################
leg_joint_names = ['left_hip_pitch', 'left_hip_roll', 'left_hip_yaw', 'left_knee_pitch',
             'left_ankle_pitch', 'left_ankle_roll', 'right_hip_pitch', 'right_hip_roll',
             'right_hip_yaw', 'right_knee_pitch', 'right_ankle_pitch', 'right_ankle_roll']
arm_joint_names = ['left_shoulder_pitch', 'left_shoulder_roll', 'left_shoulder_yaw', 'left_elbow_pitch',
                   'left_elbow_yaw', 'left_wrist_pitch', 'left_wrist_roll',
                   'right_shoulder_pitch', 'right_shoulder_roll', 'right_shoulder_yaw', 'right_elbow_pitch',
                   'right_elbow_yaw', 'right_wrist_pitch', 'right_wrist_roll']

arm_dofs_idx, leg_dofs_idx, base_dofs_idx = [], [], []

try:
    arm_dofs_idx = [robot.get_joint(name).dof_idx_local for name in arm_joint_names]
    print(f'arm_dofs_idx: {arm_dofs_idx}')
except Exception as e:
    print(e)
    
try:
    leg_dofs_idx = [robot.get_joint(name).dof_idx_local for name in leg_joint_names]
    print(f'leg_dofs_idx: {leg_dofs_idx}')
except Exception as e:
    print(e)

try:
    base_dofs_idx = [robot.get_joint('floating_base').dof_idx_local]
    print(f'base_dofs_idx: {base_dofs_idx}')
except Exception as e:
    print(e)

arm_dofs_idx: [0, 2, 4, 6, 8, 10, 12, 1, 3, 5, 7, 9, 11, 13]
Joint not found for name: left_hip_pitch.
Joint not found for name: floating_base.


In [8]:
########################## links ##########################

# link的索引从1开始
body_link_names = ['x1-body', 'body_yaw', 'body_roll', 'body_pitch']
arm_link_names = [name + '_link' for name in arm_joint_names]
leg_link_names = [name + '_link' for name in leg_joint_names]
link_names = body_link_names + arm_link_names + leg_link_names

info = []
for name in link_names:
    idx = robot.get_link(name).idx
    pos = robot.get_links_pos()[idx-1]
    info.append((idx, name, pos))

info.sort(key=lambda x: x[0])
for idx, name, pos in info:
    print(f'{idx} {name} {pos}')

1 x1-body tensor([0.0000, 0.0000, 0.6500], device='cuda:0')
2 body_yaw tensor([0.0025, 0.0000, 0.7655], device='cuda:0')
3 left_hip_pitch_link tensor([0.0025, 0.0923, 0.6379], device='cuda:0')
4 right_hip_pitch_link tensor([ 0.0025, -0.0923,  0.6379], device='cuda:0')
5 body_roll tensor([0.0025, 0.0000, 0.8060], device='cuda:0')
6 left_hip_roll_link tensor([0.0429, 0.1339, 0.5962], device='cuda:0')
7 right_hip_roll_link tensor([ 0.0429, -0.1339,  0.5962], device='cuda:0')
8 body_pitch tensor([0.0025, 0.0000, 0.8060], device='cuda:0')
9 left_hip_yaw_link tensor([0.0023, 0.1339, 0.5124], device='cuda:0')
10 right_hip_yaw_link tensor([ 0.0023, -0.1339,  0.5185], device='cuda:0')
11 left_shoulder_pitch_link tensor([0.0025, 0.1458, 1.0620], device='cuda:0')
12 right_shoulder_pitch_link tensor([ 0.0024, -0.1458,  1.0620], device='cuda:0')
13 left_knee_pitch_link tensor([0.0023, 0.1002, 0.3702], device='cuda:0')
14 right_knee_pitch_link tensor([ 0.0023, -0.1004,  0.3702], device='cuda:0')
15 

In [9]:
kv = robot.get_dofs_kv()
print(f'kv: {kv}')
kp = robot.get_dofs_kp()
print(f'kp: {kp}')

kv: tensor([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
       device='cuda:0')
kp: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       device='cuda:0')


In [10]:
############ 可选：设置控制增益 ############
# 设置位置增益
robot.set_dofs_kp(
    kp             = np.array([200, 150, 150, 120, 100, 80, 80, 200, 150, 150, 120, 100, 80, 80]),
    # dofs_idx_local = arm_dofs_idx,
)
# 设置速度增益
robot.set_dofs_kv(
    kv             = np.array([10, 8, 8, 6, 5, 4, 4, 10, 8, 8, 6, 5, 4, 4]),
    # dofs_idx_local = arm_dofs_idx,
)
# 设置安全的力范围
robot.set_dofs_force_range(
    lower          = np.array([-100, -80, -80, -50, -30, -20, -20, -100, -80, -80, -50, -30, -20, -20]),
    upper          = np.array([100, 80, 80, 50, 30, 20, 20, 100, 80, 80, 50, 30, 20, 20]),
    # dofs_idx_local = arm_dofs_idx,
)
kv = robot.get_dofs_kv()
print(f'kv: {kv}')
kp = robot.get_dofs_kp()
print(f'kp: {kp}')

kv: tensor([10.,  8.,  8.,  6.,  5.,  4.,  4., 10.,  8.,  8.,  6.,  5.,  4.,  4.],
       device='cuda:0')
kp: tensor([200., 150., 150., 120., 100.,  80.,  80., 200., 150., 150., 120., 100.,
         80.,  80.], device='cuda:0')


In [32]:
########################## end effectors ##########################
end_effector_names = ['left_wrist_roll_link', 'right_wrist_roll_link']
effector_pos = robot.get_links_pos()
end_effector_idx = [robot.get_link(name).idx_local for name in end_effector_names]
ee_pos = [effector_pos[i] for i in end_effector_idx]
print(f'end effector positions: {ee_pos}')

end effector positions: [tensor([0.2574, 0.1022, 0.9552], device='cuda:0'), tensor([ 0.2513, -0.1030,  0.9731], device='cuda:0')]


In [41]:
########################## target_pos ##########################
for i in range(100):
    scene.step()

init_pos = [[0.0026, 0.1990, 0.6439], [0.0026, -0.1990, 0.6439]]

box_pos = box.get_pos()
target_pos = [[box_pos[0], box_pos[1]+box_size/2, box_pos[2]], 
              [box_pos[0], box_pos[1]-box_size/2, box_pos[2]]]

# target_pos = [[0.3000, 0., 1.0000], [0.3000, -0.15, 1.0000]]

box_pos

tensor([2.0000e-01, 3.0172e-07, 8.9991e-01], device='cuda:0')

In [42]:
########################## inverse_kinematics ##########################
ee_link = [robot.get_link(name) for name in end_effector_names]
qpos = robot.inverse_kinematics_multilink(ee_link, target_pos)
qpos

tensor([-2.2735e-01,  2.2819e-01,  0.0000e+00,  0.0000e+00,  4.0616e-01,
        -4.0583e-01,  1.7563e+00, -1.7535e+00,  1.4376e-01, -2.9675e-01,
         6.2653e-04,  7.5127e-04, -1.6211e-04, -1.6209e-04], device='cuda:0')

In [43]:
########################## path planning ##########################
path = robot.plan_path(
    qpos_goal        = qpos,
    num_waypoints    = int(t / scene.dt), # 2秒时长
    ignore_collision = True,
)

# 执行规划路径
for waypoint in path:
    robot.control_dofs_position(waypoint)
    scene.step()

# 等待到达最后一个路径点
for i in range(100):
    # print('控制力:', robot.get_dofs_control_force(arm_dofs_idx))
    # print('内部力:', robot.get_dofs_force(arm_dofs_idx))
    scene.step()

In [44]:
########################## hold ##########################
should_pith_names = ['left_shoulder_pitch', 'right_shoulder_pitch']
should_pitch_idxs = [robot.get_joint(name).dof_idx_local for name in should_pith_names]
should_pitch_idxs = torch.tensor(should_pitch_idxs)

should_roll_names = ['left_shoulder_roll', 'right_shoulder_roll']
should_roll_idxs = [robot.get_joint(name).dof_idx_local for name in should_roll_names]
should_roll_idxs = torch.tensor(should_roll_idxs)

shoulder_yaw_names = ['left_shoulder_yaw', 'right_shoulder_yaw']
shoulder_yaw_idxs = [robot.get_joint(name).dof_idx_local for name in shoulder_yaw_names]
shoulder_yaw_idxs = torch.tensor(shoulder_yaw_idxs)

elbow_pitch_names = ['left_elbow_pitch', 'right_elbow_pitch']
elbow_pitch_idxs = [robot.get_joint(name).dof_idx_local for name in elbow_pitch_names]
elbow_pitch_idxs = torch.tensor(elbow_pitch_idxs)

mask = ~torch.isin(torch.tensor(arm_dofs_idx), should_pitch_idxs)
hold_idxs = torch.tensor(arm_dofs_idx)[mask]

# # 用torque控制指定自由度，其余的用位置控制。左右对称
# robot.control_dofs_force(torch.tensor([10, -10]), should_pitch_idxs)
robot.control_dofs_position(torch.tensor([0.2, -0.2]), should_pitch_idxs)
dof_pos = robot.get_dofs_position(hold_idxs)
dof_pos[6:] = -dof_pos[:6]
robot.control_dofs_position(dof_pos[mask], hold_idxs)
for i in range(100):
    scene.step()

In [46]:
########################## reset ##########################
for i in range(100):
    robot.control_dofs_position(torch.zeros(robot.n_dofs))
    scene.step()

In [47]:
########################## completely reset ##########################
robot.set_dofs_position(torch.zeros(robot.n_dofs), arm_dofs_idx)
box.set_dofs_position(torch.tensor([0.2, 0. , 1. , 0. , 0. , 0. ]), box.base_joint.dof_idx_local)
scene.step()

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def quaternion_to_euler(quaternion):
    """
    将四元数转换为欧拉角（单位：弧度）。
    参数：
    quaternion (array-like): 四元数 [x, y, z, w]
    返回：
    euler_angles (array): 对应的欧拉角 [roll, pitch, yaw]（单位：弧度）
    """
    # 创建旋转对象
    r = R.from_quat(quaternion)
    # 获取欧拉角 (yaw, pitch, roll)，使用 ypr 顺序
    euler_angles = r.as_euler('zyx', degrees=False)
    return torch.tensor(euler_angles, device=device)

quaternion = [1, 0, 0,  1.0000e+00]  # 例如一个四元数
euler_angles = quaternion_to_euler(quaternion)
print(euler_angles)


tensor([0.0000, 0.0000, 1.5708], device='cuda:0', dtype=torch.float64)


In [19]:
origin_qpos = robot.get_qpos()
print(f'origin_qpos: {origin_qpos}')
arm_dof_pos = robot.get_dofs_position(arm_dofs_idx)
print(f'arm_dof_pos: {arm_dof_pos}')
if len(base_dofs_idx) > 0:
    origin_dof_pos = robot.get_dofs_position(base_dofs_idx[0])
    print(f'origin_dof_pos: {origin_dof_pos}')
    origin_q_euler = quaternion_to_euler(origin_qpos[3:7].cpu())
    print(f'origin_q_euler: {origin_q_euler}')

origin_qpos: tensor([ 0.0080, -0.0209, -0.0010, -0.0185, -0.0472,  0.0419, -0.0079,  0.0673,
         0.0575,  0.0874,  0.0111, -0.0514,  0.0398,  0.0417], device='cuda:0')
arm_dof_pos: tensor([ 0.0080, -0.0010, -0.0472, -0.0079,  0.0575,  0.0111,  0.0398, -0.0209,
        -0.0185,  0.0419,  0.0673,  0.0874, -0.0514,  0.0417], device='cuda:0')


In [20]:
# 通过set_dofs_position暴力执行
if len(base_dofs_idx) > 0:
    q_euler = quaternion_to_euler(qpos[3:7])
    qpos_euler = torch.cat([qpos[:3], q_euler, qpos[7:]])
else:
    qpos_euler = qpos
robot.set_dofs_position(qpos_euler)

if len(base_dofs_idx) > 0:
    dof_euler = robot.get_dofs_position(base_dofs_idx[0])[3:]
    print(f'q_euler: {q_euler}')
    print(f'dof_euler: {dof_euler}')
print(f'qpos: {qpos}')
print(f'dofs_position: {robot.get_dofs_position(arm_dofs_idx)}')

qpos: tensor([ 0.4058, -0.4064,  0.0000,  0.0000,  0.4394, -0.4392,  1.1410, -1.1459,
         0.3512, -0.1611,  0.0693,  0.0686, -0.0123, -0.0122], device='cuda:0')
dofs_position: tensor([ 0.4058,  0.0000,  0.4394,  1.1410,  0.3512,  0.0693, -0.0123, -0.4064,
         0.0000, -0.4392, -1.1459, -0.1611,  0.0686, -0.0122], device='cuda:0')


In [21]:
for i in range(robot.n_dofs):
    if i in arm_dofs_idx:
        print(f'{i} arm {qpos[i+1]}')
    elif i in leg_dofs_idx:
        print(f'{i} leg {qpos[i+1]}')
    else:
        print(f'{i} base {qpos[i+1]}')

0 arm -0.40643084049224854
1 arm 0.0
2 arm 0.0
3 arm 0.4394027292728424
4 arm -0.4392257630825043
5 arm 1.1410330533981323
6 arm -1.1459113359451294
7 arm 0.35118940472602844
8 arm -0.16112135350704193
9 arm 0.06933190673589706
10 arm 0.06864558160305023
11 arm -0.012252932414412498
12 arm -0.012247169390320778


IndexError: index 14 is out of bounds for dimension 0 with size 14